<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/SVD(SingularVectorDecomposition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surprise 패키지를 활용한 SVD 모델 구현

## SVD 모델의 원리

Singular Vector Decomposition을 이용한 모델.

singular matrix = 역행렬이 없는 매트릭스를 의미
det(A) = 0

```
a = [[a,b],
     [c,d]]
일때,
det(A) = ad - bc

A^-1 = det{A}^-1
```

A: 기존 Input data matrix [m x n] 행렬

U: 왼쪽 singular matrix -> y에 따라 x가 무한이거나 존재하지 않음

$\sum$: r개의 특이값을 갖는 대각행렬

V: 오른쪽 singluar matrix

특이값(Singular Value): m x n 의 행렬 A가 정방일때 이를 $A^TA$한 대칭행렬의 고유값을 구해 루트를 씌워준 값을 의미

고유값(Eigen Value): 임의의 벡터 v에 선형변환 A를 적용했을 때, Av = $\lambda$v를 만족하는 상황에서의 $\lambda$값

고유벡터(Eigen Vector): 위 상황에서의 v값을 의미하며, 고유값을 구한 뒤, 가우스 소거법을 이용해 계산한다.







In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 28.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630164 sha256=ec98c8fc9bca2f0ed5ab7b7bf4bb9c310bb29f38b245b6fa414ce3e9894cc014
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt = False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
print(data)

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data[:10], measures=['rmse', 'mae'], cv = 5, verbose = True)

TypeError: ignored